# Getting started with DSPy on Databricks 
In this notebook, we'll go into **how** of using DSPy to build a non-optimized RAG app on Databricks, in the process of which we'll look at **why** we should use DSPy for building GenAI solutions on Databricks. 

### Install and import packages

In [0]:
%pip install -U dspy mlflow databricks-agents databricks-sdk==0.50.0 
dbutils.library.restartPython()

In [0]:
import dspy
import math
import mlflow
import pandas as pd
from mlflow.models import ModelConfig

from dspy.retrieve.databricks_rm import DatabricksRM

# from databricks.agents.evals import generate_evals_df
# from databricks.vector_search.client import VectorSearchClient



### Load model config

In [0]:
config_file = "config.yaml"
model_config = ModelConfig(development_config=config_file)

### Set up variables

In [0]:
CATALOG = model_config.get("catalog")
SCHEMA = model_config.get("schema")

VECTOR_SEARCH_ENDPOINT = model_config.get("vector_search_endpoint")
VECTOR_SEARCH_INDEX = model_config.get("vector_search_index")
index_path = f"{CATALOG}.{SCHEMA}.{VECTOR_SEARCH_INDEX}"

model = model_config.get("chat_endpoint_name")

LM = f"databricks/{model}"


## Why DSPy?
You can build an LLM application without any frameworks, but using a framework reduces the amount of effort it takes to build an application.

At a high level, all LLM frameworks offer some abstractions to help you modularize your application code. They provide interfaces for connecting to different LMs, tools & other services e.g. retrievers, etc.  
 
While DSPy offers similar benefits to other LLM frameworks, it stands out in a few ways.

The premise of DSPy is that it focuses on programming rather than prompting. This violates the mental model we all have of LLMs, because we're so used to interacting with LLMs using natural language. 

While this works really well for simple QnA that most of do with chatGPT daily, anyone who has built and put a GenAI application into production will know how incredibly time consuming developing and iterating on the prompt is. It becomes even more so when you want to compare different models, each of which may have different prompting guidelines.   

One of the key that DSPy stands out from other frameworks is that it goes one step further than the other frameworks, by offering an abstraction for prompt engineering. 

This means that DSPy allows you to focus on the logic rather than language.

By writing code instead of writing prompts:
1. You focus on the logic rather than on forcing the model to behave a certain way
2. It's easier and more intuitive to iterate and expand your implementation, e.g. add another output field

### Signatures

To understand how to do this, we first need to understand **[Signatures](https://dspy.ai/learn/programming/signatures/?h=signature)**. 

At a high level, the purpose of the Signature is to define the inputs and the outputs required from your program. By creating a Signature, you're taking a declarative approach to prompt engineering. One way to see the value of this is to think about the Databricks vision with DLT. Similarly to how DLT simplifies ETL by providing a declarative approach, DSPy simplifies GenAI workflows by providing a declarative approach. The same way a really experienced Spark engineer could spend a lot of time on building a pipeline which is as performant as DLT, a seasoned prompt engineer could invest time and energy into building a very good prompt, which may be as good as the one created by DSPy, but that comes at a cost of time and effort. 

The docs linked above go into a lot of detail on Signatures. Because we're looking at a simple RAG implementation, we'll be using an inline Signature in our example. However, you also have the option for creating class based Signatures which allow you to clarify the task in the docstring, and provide hints regarding the input or output fields. The key thing to remember is your Signature needs to be explicit about **what** you want your model to do, not **how**. Once you make this mental switch from imperative (prompting) to declarative (Signatures), writing DSPy programs becomes second nature.

### Modules

After writing your signature, the next step is to create a **[Module](https://dspy.ai/learn/programming/modules/)**. Modules are the building blocks of your programs. You want to consider having a Module for each logically independent component of your solution. DSPy provides some built-in Modules which you will typically compose into your own custom ones, as in the example below. Your custom Modules always need an `__init__` method and a `forward` method. 

## RAG Module
Let's start building a RAG module with DSPy. The syntax below with `dspy.Module` allows you to connect the pieces we need for a RAG app together. These are, our **retriever** (which is a wrapper around our Databricks vector search index) and a **generator** which itself uses one of DSPy's built-in Modules to generate the response based on the question and the retrieved context. 

Concretely, in the `__init__` method, you declare any sub-module you'll need, which in this case is just a `dspy.Predict('context, question -> response')` module that takes retrieved context, a question, and produces a response. In the `forward` method, you simply express any Python control flow you like, using your modules. In this case, we first invoke our retriever followed by `self.response_generator`.

#### Note
One key thing to note is that we've used the flag `for_mosaic_agent_true` set to True. We use this in 2 places - first we pass it to the retriever parameter `use_with_databricks_agent_framework`. This is needed to ensure the response object is a list of dictionaries compatible with Databricks agents. Without the flag the RM produces a [Prediction](https://dspy.ai/api/primitives/Prediction/) object. Additionally, we also use our flag to help specify how to parse the inputs when received as a ChatAgentMessage or a ChatModelMessage in the `forward` function.

In [0]:
class RAG(dspy.Module):
  def __init__(self, for_mosaic_agent=True): 
    # setup mlflow tracing
    mlflow.dspy.autolog()

    # setup flag indicating if the object will be deployed as a Mosaic Agent
    self.for_mosaic_agent = for_mosaic_agent
    self.lm = dspy.LM(LM, cache=False)

    # setup the primary retriever pointing to the chunked documents
    self.retriever = DatabricksRM(
        databricks_index_name=index_path,
        text_column_name="page_content",
        docs_id_column_name="unique_chunk_index",
        columns=["page_content"],
        k=5,
        use_with_databricks_agent_framework=for_mosaic_agent
    )
    
    self.response_generator = dspy.Predict("context, question -> response")

  def forward(self, question):
    if self.for_mosaic_agent:
      # When using a mosaic agent, questions can be of type ChatAgentMessage or (TODO: accept ChatModelMessage) 
      question = question["messages"][-1]["content"]

    context = self.retriever(
        question
    )

    with dspy.context(lm=self.lm):
      response = self.response_generator(context=context, question=question)

      if self.for_mosaic_agent:
        return response.response
      return response

## GenAI Tracing via MLflow Autologging

You'll notice on line 4 of cell 11, we've used `mlflow.dspy.autolog()`

MLflow Tracing is a feature that enhances observability in Generative AI applications by capturing detailed information about the execution of services. It records inputs, outputs, and metadata for each step of a request, aiding in debugging and performance monitoring. Tracing is an incredibly valuable way to understand how and where to improve on your programs, as well as understand what's happening under the hood with DSPy. By simply using `mlflow.dspy.autolog()` you get detailed execution traces for you DSPy GenAI applications. 

### Testing
Lets test our RAG module and look at the mlflow trace in the notebook.

In [0]:
rag = RAG()

In [0]:
result = rag({'messages': [{'content': 'What features are included in the "Machine learning tutorial" notebook for the scikit-learn package?', 'role': 'user'}]})

#### Under the hood
The MLFlow trace helps us understand the steps DSPy took under the hood. Lets walk through the trace above to understand what happened to return the response we received. 

The trace shows the steps taken in *reverse order*, i.e. the last step is at the top of the trace. 

1. At the top level we have `RAG.forward`, in which you can see the inputs which were provided, and the outputs returned. 
2. This is followed by `DatabricksRM.forward`, the forward method of the `DatabricksRM` module we used to query our vector search index. This shows you the query sent to vector search and the documents and metadata returned.
3. Finally, we have `Predict.forward`, the forward method of the `Predict` module we configured with the `Signature` we wrote. You can see the variable names we specified in our `Signature` defined in the Inputs and Outputs of the module. Within `Predict.forward` you'll notice 3 sub methods:
    1. `ChatAdapter.format` - **[Adapters](https://dspy.ai/api/adapters/Adapter/)** are a layer within DSPy which handle structuring inputs and parsing structured outputs to fit your Signature.  
    2. `LM.__call__` - the LLM call - here you can see the system prompt created by the DSPy module, the retrieved documents sent along with the prompt, and the response from the LLM. 
    3. `ChatAdapter.parse` - another Adapter method which at this point is formatting the response to match the outputs defined in the Signature. 

## Evaluation
While we can see DSPy has done a lot of work for us, we often need to measure the quality of our solution. While DSPy offers it's own **[Evaluation](https://dspy.ai/learn/evaluation/overview/)** methods, which you can leverage within Databricks, in this notebook we're going to leverage Mosaic's AI Agent Evaluation to see how it works seamlessly with DSPy. 

Since we don't have a curated dataset to evaluate against, we're going to start by creating and saving a synthetic evaluation dataset. 

In [0]:
eval_table = model_config.get("eval_table")
eval_table_path = f"{CATALOG}.{SCHEMA}.{eval_table}"

#### Create synthetic evaluation dataset and save to table
- Uncomment cells 20 & 21 if you're running this notebook for the first time.
- Comment cells 20 & 21 for subsequent runs when you want to reference the same evaluation dataset

In [0]:
##### Uncomment this cell if you're running the notebook for the first time. 
# docs_table = model_config.get("table")
# docs_table_path = f"{CATALOG}.{SCHEMA}.{docs_table}"

# docs_df = spark.read.table(docs_table_path).selectExpr("page_content as content", "source as doc_uri")
# display(docs_df)

In [0]:
##### Uncomment this cell if you're running the notebook for the first time
# agent_description = """ 
# The agent is a RAG chatbot that answers questions about Databricks. The Agent has access to a corpus of Databricks documents, and its task is to answer the user's questions by retrieving the relevant docs from the corpus and synthesizing a helpful, accurate response.
# """

# question_guidelines = """
# # User personas
# - A developer who is new to the Databricks platform
# - An experienced, highly technical Data Scientist or Data Engineer

# # Example questions
# - what API lets me parallelize operations over rows of a delta table?
# - Which cluster settings will give me the best performance when using Spark?

# # Additional Guidelines
# - Questions should be succinct, and human-like
# """

# num_evals = 100

# evals = generate_evals_df(
#     docs_df,
#     num_evals=num_evals,
#     agent_description=agent_description

# )

# eval_df = spark.createDataFrame(evals)

# eval_df.write.format("delta").mode("overwrite").saveAsTable(eval_table)

#### Load synthetic eval table

In [0]:
eval_df = spark.table(eval_table_path)
display(eval_df)

#### Run evaluation

We simply pass in the `rag` module we created and our evaluation dataset to the mlflow `evaluate` call, and we can see the results in the mlflow run under "traces". 

Please note that this is an engineered example and the focus of this notebook is not on the quality of the results (the vector search is quite poor for now!) but to show the dev flow with DSPy and the integration with Databricks native capabilities. 

In [0]:
# TODO: https://docs.databricks.com/aws/en/generative-ai/agent-evaluation/evaluate-agent#option-4-local-function-in-the-notebook

mlflow.evaluate(
  model=rag,
  data=eval_df,
  model_type="databricks-agent"
)

## Summary
We've looked at how to:
1. Write a Signature
2. Create a module
3. Use DSPy with Databricks' and Mosaic capabilities:
  1. Vector search
  2. MLflow tracing
  3. Mosaic Agent Eval